In [1]:
try:
  import ipywidgets as widgets
except ImportError:
  widgets=None

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

In [3]:
fig = plt.figure(figsize=(5,4))
ax=plt.subplot(111)
plt.close(fig)

In [4]:
# pseudocode 2D-LIF

# for each time period t:
  # update V from value at time t to time (t+dt)
  # update Vt (threshold) by constant delta
  # process incoming spikes- modify V by synaptic weight w
  # check if V crossed threshold Vt
  # if so:
    # emit spike
    # increment Vt by dt
    # reset V

In [5]:
# function to run simulation
def LIF(tau=10, # tau = time constant
        t0=10,t1=40,t2=60, # t0,t1,t2 = time of 3 input spikes
        w=0.1, # w = input synapse weight
        threshold=1.0, # threshold value to produce spike
        dt_threshold=0.5, # threshold dynamic constant
        reset=0.0): #reset value after spike
  times = [t0,t1,t2]  # spike times
  times.sort(reverse=True) #sort times- more efficient to take the last value off the list

  # set parameters
  duration = 100 # time total in ms
  dt=0.1 # timestamp in ms
  V=0.0 # initial membrane potential
  Vt=threshold

  V_rec=[] # list to record membrane potentials
  Vt_rec=[] # list to record threshold values
  spikes = [] # list to store spike times
  T=np.arange(np.round(duration/dt))*dt # array of times

  alpha=np.exp(-dt/tau) # factor V decays each time step
  delta=np.exp(-dt/tau) # factor Vt decays each time step

  # generate plot
  ax.clear()
  for t in times:
    ax.axvline(t,ls='--',c='b') # blue dashed line per spike (NOT WORKINGG)

  # run the simulation
  for t in T:
    V_rec.append(V) # record V
    Vt_rec.append(Vt) # record Vt
    V*=alpha #integrate equations
    Vt=(Vt-threshold)*delta+threshold # increase by threshold constant delta Vt
    if times and t>times[-1]: # if there is input spike
      V+=w # modify V by snaptic weight w
      times.pop() # remove spike from list
    V_rec.append(V) # record V before reset to see spike
    Vt_rec.append(Vt) # record Vt

    if V>Vt: # is there an output spike? (fired)
      V=reset
      Vt+=dt_threshold # increment Vt (threshold) by dt
      spikes.append(t)

  # generate complete plot (?)
  ax.plot(np.repeat(T,2),V_rec,'-k',lw=2)
  ax.plot(np.repeat(T,2),Vt_rec,'--g',lw=2)
  for t in spikes:
    ax.axvline(t,ls='--',c='r') # red dashed line per firing spike (crosses thresh)
  # set axes
  ax.set_xlim(0,duration)
  ax.set_xlabel('Time (ms)')
  ax.set_ylim(-1,2)
  ax.set_ylabel('V')

  plt.tight_layout()
  display(fig);

# create interactive widget for variables
widgets.interact(LIF,
    tau=widgets.IntSlider(min=1,max=100,values=50),
    t1=widgets.IntSlider(min=0,max=100,values=20),
    t0=widgets.IntSlider(min=0,max=100,values=40),
    t2=widgets.IntSlider(min=0,max=100,values=60),
    w=widgets.FloatSlider(min=-1,max=2,step=0.05,value=0.5),
    threshold=widgets.FloatSlider(min=0.0,max=2.0,step=0.05,values=1.0),
    dt_threshold=widgets.FloatSlider(min=0.0,max=1.0,step=0.05,values=0.5),
    reset=widgets.FloatSlider(min=0.0,max=1.0,step=0.05,value=0.0)
    );

interactive(children=(IntSlider(value=1, description='tau', min=1), IntSlider(value=0, description='t0'), IntS…